## Ucitavanje i Sredjivanje Nizova Podataka

In [1]:
#ucitavanje potrebnih biblioteka
import pandas as pd
import numpy as np

In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("epl-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("epl-2024.csv")
#sezona 25/6
link_25 = pd.read_html("https://fixturedownload.com/results/epl-2025")
df_25_6 = link_25[0]


## Ukidanje Duplih Vrijednosti za Nazive Klubova

In [3]:
#za sezonu 25 6
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')
#za sezonu 24 5
df_24_5["Home Team"] = df_24_5["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_24_5["Away Team"] = df_24_5["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')

## Određivanje Duzine Trenutnoj Sezoni i Prikaz

In [4]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6[df_25_6["Result"] != "-"]
df_25_6.tail(5)

,Round Number,Date,Location,Home Team,Away Team,Result
165,17,20/12/2025 17:30,Tottenham Hotspur Stadium,Spurs,Liverpool,1 - 2
166,17,20/12/2025 20:00,Hill Dickinson Stadium,Everton,Arsenal,0 - 1
167,17,20/12/2025 20:00,Elland Road,Leeds,Crystal Palace,4 - 1
168,17,21/12/2025 16:30,Villa Park,Aston Villa,Man Utd,2 - 1
169,17,22/12/2025 20:00,Craven Cottage,Fulham,Nottingham Forest,1 - 0


In [5]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)

In [6]:
df_BL["Home Team"].unique()

array(['Burnley', 'Arsenal', 'Bournemouth', 'Brighton', 'Everton',
       'Sheffield Utd', 'Newcastle', 'Brentford', 'Chelsea', 'Man Utd',
       'Nottingham Forest', 'Fulham', 'Liverpool', 'Wolves', 'Spurs',
       'Man City', 'Aston Villa', 'West Ham', 'Crystal Palace', 'Luton',
       'Ipswich', 'Leicester', 'Southampton', 'Sunderland', 'Leeds'],
      dtype=object)

In [7]:
#prikaz sjedinjenog niza podatak 
#tj. sve tri sezone u jednom nizu podataka u obliku df 
df_BL.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
165,NaN,17,20/12/2025 17:30,Tottenham Hotspur Stadium,Spurs,Liverpool,1 - 2
166,NaN,17,20/12/2025 20:00,Hill Dickinson Stadium,Everton,Arsenal,0 - 1
167,NaN,17,20/12/2025 20:00,Elland Road,Leeds,Crystal Palace,4 - 1
168,NaN,17,21/12/2025 16:30,Villa Park,Aston Villa,Man Utd,2 - 1
169,NaN,17,22/12/2025 20:00,Craven Cottage,Fulham,Nottingham Forest,1 - 0


## Predprocesovanje Niza Podataka

In [8]:
#Izbacivanje Nerelevantnih Osobina
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.shape

(930, 3)

In [9]:
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

In [10]:
df_BL.to_csv("epl.csv")
df_BL

,H,A,Result
0,Burnley,Man City,0 - 3
1,Arsenal,Nottingham Forest,2 - 1
2,Bournemouth,West Ham,1 - 1
3,Brighton,Luton,4 - 1
4,Everton,Fulham,0 - 1
...,...,...,...
165,Spurs,Liverpool,1 - 2
166,Everton,Arsenal,0 - 1
167,Leeds,Crystal Palace,4 - 1
168,Aston Villa,Man Utd,2 - 1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

#One Hot Encoder Timova - Domacin i Gost
onehot_encoder_domacin = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_domacin = onehot_encoder_domacin.fit_transform(df_BL[["H"]]).toarray()
df_domacin = pd.DataFrame(kodiran_domacin,columns=onehot_encoder_domacin.get_feature_names_out(["H"]))

In [12]:
df_domacin.columns

Index(['H_Aston Villa', 'H_Bournemouth', 'H_Brentford', 'H_Brighton',
       'H_Burnley', 'H_Chelsea', 'H_Crystal Palace', 'H_Everton', 'H_Fulham',
       'H_Ipswich', 'H_Leeds', 'H_Leicester', 'H_Liverpool', 'H_Luton',
       'H_Man City', 'H_Man Utd', 'H_Newcastle', 'H_Nottingham Forest',
       'H_Sheffield Utd', 'H_Southampton', 'H_Spurs', 'H_Sunderland',
       'H_West Ham', 'H_Wolves'],
      dtype='object')

In [13]:
#One Hot Encoder Timova - Gosti
onehot_encoder_gost = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_gost = onehot_encoder_gost.fit_transform(df_BL[["A"]]).toarray()
df_gost = pd.DataFrame(kodiran_gost,columns=onehot_encoder_gost.get_feature_names_out(["A"]))

In [14]:
#Sjedinjavanje tih osobina domacin i gost
df_jedan = pd.concat([df_domacin,df_gost],axis=1)
df_jedan

,H_Aston Villa,H_Bournemouth,H_Brentford,H_Brighton,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Fulham,H_Ipswich,...,A_Man City,A_Man Utd,A_Newcastle,A_Nottingham Forest,A_Sheffield Utd,A_Southampton,A_Spurs,A_Sunderland,A_West Ham,A_Wolves
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
928,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Transformisanje Rezultata u Varijablu - Y Varijable

#### Stvaranje Vrijednosti za Dvije Kolone iz Jedne Osobine/Kolone Rezultat

In [15]:
rez_dom = df_BL["Result"].str.slice(0,1)

In [16]:
rez_dom.head()

0    0
1    2
2    1
3    4
4    0
Name: Result, dtype: object

In [17]:
rez_gost = df_BL["Result"].str.slice(4,5)

In [18]:
rez_gost.head()

0    3
1    1
2    1
3    1
4    1
Name: Result, dtype: object

In [19]:
df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)

In [20]:
#promjena naziva kolonama - spajanjem smo dobili kolone istih naziva
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]
df_rezultatski.tail()

,H,A,Result,Rez1,Rez2
165,Spurs,Liverpool,1 - 2,1,2
166,Everton,Arsenal,0 - 1,0,1
167,Leeds,Crystal Palace,4 - 1,4,1
168,Aston Villa,Man Utd,2 - 1,2,1
169,Fulham,Nottingham Forest,1 - 0,1,0


In [21]:
#spremanje df_epl varijable za obradu osobine Result u excelu
df_rezultatski.to_csv("uredjivanje_skora_vani.csv")

## Uocavanje Indeksa Sezona - za Kasnije Floatiranje

In [22]:
#df_rezultatski[0:380] #sezona 23/4 od 0:380
#df_rezultatski[380:759] #sezona 24/5 od 380:759
#df_rezultatski[760:] #sezona25/6 od 760:

## Stvaranje Kolone Skor sa Vrijednostima - Ovdje

In [23]:
# Define a function to perform the conditional logic
def calculate_value(row):
    if row['Rez1'] > row['Rez2']:
        return 1
    elif row['Rez1'] < row['Rez2']:
        return -1
    else:  # row['Column1'] == row['Column2']
        return 0

In [24]:
# Apply the function to each row in the DataFrame and assign the result to a new column
df_rezultatski['Skor'] = df_rezultatski.apply(calculate_value, axis=1)
skor = df_rezultatski.drop(['H', 'A', 'Result', 'Rez1',"Rez2"],axis=1)

### Floatiranje Vrijednosti Kolone Skor - po Sezonama

In [25]:
#sezona 23/4
skor23_4 = skor[0:380]
skor24_5 = skor[380:760]
skor25_6 = skor[760:]

In [26]:
#skor.head(3)
skor23_4["Skor"] = skor23_4["Skor"] * 0.71

C:\Users\amirb\AppData\Local\Temp\ipykernel_13668\3544292283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor23_4["Skor"] = skor23_4["Skor"] * 0.71


In [27]:
skor24_5["Skor"] = skor24_5["Skor"] * 0.85

C:\Users\amirb\AppData\Local\Temp\ipykernel_13668\3136881136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor24_5["Skor"] = skor24_5["Skor"] * 0.85


## Konačna kolona Skor - Sjedinjavanje Vrijednosti Sezona

In [28]:
skor = pd.concat([skor23_4, skor24_5, skor25_6], axis=0, ignore_index=True)

In [29]:
df_rezultatski = df_rezultatski.reset_index(drop=True)
skor = skor.reset_index(drop=True)

In [30]:
#provjera rezultata
finalna_provjera = pd.concat([df_rezultatski,skor],axis=1,ignore_index=True)
finalna_provjera = finalna_provjera.drop([3,4,5],axis=1)
finalna_provjera.columns = ["H","A","Rezultat","Skor"]

In [31]:
finalna_provjera[-5:-1]

,H,A,Rezultat,Skor
925,Spurs,Liverpool,1 - 2,-1.0
926,Everton,Arsenal,0 - 1,-1.0
927,Leeds,Crystal Palace,4 - 1,1.0
928,Aston Villa,Man Utd,2 - 1,1.0


# Konačni df - Podaci su Sređeni

In [32]:
skor = skor.reset_index(drop=True)
df_jedan = df_jedan.reset_index(drop=True)

In [33]:
df_BL = pd.concat([df_jedan,skor],axis=1)

In [34]:
df_BL.head(5)

,H_Aston Villa,H_Bournemouth,H_Brentford,H_Brighton,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Fulham,H_Ipswich,...,A_Man Utd,A_Newcastle,A_Nottingham Forest,A_Sheffield Utd,A_Southampton,A_Spurs,A_Sunderland,A_West Ham,A_Wolves,Skor
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71


In [35]:
#spremanje kodiranog niza podataka
df_BL.to_csv("kodirani_epl.csv")

## Razdvajanje Niza Podataka na Osobine za Predvidjanje i Ciljanu Osobinu

In [36]:
# X predvidjacke, y varijabla jeste Skor Osobina
X = df_BL.drop(["Skor"],axis=1)
y = df_BL["Skor"]

In [37]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_train.head()

,H_Aston Villa,H_Bournemouth,H_Brentford,H_Brighton,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Fulham,H_Ipswich,...,A_Man City,A_Man Utd,A_Newcastle,A_Nottingham Forest,A_Sheffield Utd,A_Southampton,A_Spurs,A_Sunderland,A_West Ham,A_Wolves
60,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 4.19523539, -0.23836565, -0.22402724, ..., -0.09823533,
        -0.23836565, -0.22695272],
       [-0.23836565,  4.19523539, -0.22402724, ..., -0.09823533,
        -0.23836565, -0.22695272],
       [-0.23836565, -0.23836565, -0.22402724, ..., -0.09823533,
        -0.23836565, -0.22695272],
       ...,
       [-0.23836565, -0.23836565, -0.22402724, ..., -0.09823533,
        -0.23836565, -0.22695272],
       [-0.23836565, -0.23836565, -0.22402724, ..., -0.09823533,
        -0.23836565, -0.22695272],
       [-0.23836565, -0.23836565, -0.22402724, ..., -0.09823533,
        -0.23836565, -0.22695272]])

In [39]:
#Spremanje tog Kodera - za Kasniju Upotrebu

with open("onehot_encoder_domacin.pkl","wb") as file:
    pickle.dump(onehot_encoder_domacin,file)
    
with open("onehot_encoder_gost.pkl","wb") as file:
    pickle.dump(onehot_encoder_gost,file)
    
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

## Stvaranje Modela i Dobijanje Predikcije

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense

In [41]:
#konstrukcija modela
model=Sequential([
    Dense(16,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(1,activation="tanh")
])

In [42]:
#Oblikovanje Modela 
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()

# Keras dokumentacija pojasnjava sta se dogodi sa razlicitim odabirom "loss" i drugih parametara

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                784       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [43]:
import datetime
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [44]:
#TRENIRANJE MODELA / Tovarenje Modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=50,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
27/27 [==============================] - 9s 63ms/step - loss: 0.9037 - mae: 0.9037 - val_loss: 0.8034 - val_mae: 0.8034
Epoch 2/50
27/27 [==============================] - 0s 12ms/step - loss: 0.8336 - mae: 0.8336 - val_loss: 0.7478 - val_mae: 0.7478
Epoch 3/50
27/27 [==============================] - 1s 21ms/step - loss: 0.7711 - mae: 0.7711 - val_loss: 0.7067 - val_mae: 0.7067
Epoch 4/50
27/27 [==============================] - 0s 12ms/step - loss: 0.7032 - mae: 0.7032 - val_loss: 0.6660 - val_mae: 0.6660
Epoch 5/50
27/27 [==============================] - 0s 11ms/step - loss: 0.6475 - mae: 0.6475 - val_loss: 0.6257 - val_mae: 0.6257
Epoch 6/50
27/27 [==============================] - 0s 14ms/step - loss: 0.6018 - mae: 0.6018 - val_loss: 0.6001 - val_mae: 0.6001
Epoch 7/50
27/27 [==============================] - 0s 18ms/step - loss: 0.5703 - mae: 0.5703 - val_loss: 0.5855 - val_mae: 0.5855
Epoch 8/50
27/27 [==============================] - 0s 14ms/step - loss: 0.5483 - m

In [45]:
#Procjena Rada Modela preko test podataka
test_loss, test_mae = model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 9ms/step - loss: 0.5554 - mae: 0.5554


In [46]:
#spremanje modela
model.save("epl_float_model.h5")

In [47]:
#moze biti da skaliranje je neophodno, onda je potrebno i obnoviti test parametara modela, te iznova
#-trenirati model i na kraju ga ponovo spremiti.

# ODREDI FORMU TIMOVA

In [48]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("epl-2023.csv")
#sezona 24/25
df_24_5= pd.read_csv("epl-2024.csv")
#sezona 25/6
link_25 = pd.read_html("https://fixturedownload.com/results/epl-2025")
df_25_6 = link_25[0]

In [49]:
#ukidanje duplih vrijednosti
#za sezonu 25 6
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')
#za sezonu 24 5
df_24_5["Home Team"] = df_24_5["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_24_5["Away Team"] = df_24_5["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')

In [50]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6[df_25_6["Result"] != "-"]
df_25_6.tail(5)

,Round Number,Date,Location,Home Team,Away Team,Result
165,17,20/12/2025 17:30,Tottenham Hotspur Stadium,Spurs,Liverpool,1 - 2
166,17,20/12/2025 20:00,Hill Dickinson Stadium,Everton,Arsenal,0 - 1
167,17,20/12/2025 20:00,Elland Road,Leeds,Crystal Palace,4 - 1
168,17,21/12/2025 16:30,Villa Park,Aston Villa,Man Utd,2 - 1
169,17,22/12/2025 20:00,Craven Cottage,Fulham,Nottingham Forest,1 - 0


In [51]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

In [52]:
#definisanje funkcije...
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada Te Funkcije

In [53]:
klub('Leeds')

0.2

In [54]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

rezultati = {}

#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezultat Forme

In [55]:
#Izracun
df_forme 

,Klub,Rezultat
0,Burnley,-0.8
1,Arsenal,0.4
2,Bournemouth,-0.4
3,Brighton,-0.2
4,Everton,-0.2
5,Sheffield Utd,-1.0
6,Newcastle,0.2
7,Brentford,0.0
8,Chelsea,0.0
9,Man Utd,0.2


In [56]:
#Ponovna provjera vrijednosti...
klub("Brighton")

-0.2

In [57]:
df_forme.to_csv("forma_h.csv")

In [58]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [59]:
df_forme


,Klub,Rezultat
0,Burnley,0.8
1,Arsenal,-0.4
2,Bournemouth,0.4
3,Brighton,0.2
4,Everton,0.2
5,Sheffield Utd,1.0
6,Newcastle,-0.2
7,Brentford,-0.0
8,Chelsea,-0.0
9,Man Utd,-0.2


In [60]:
df_forme.to_csv("forma_a.csv")

Kraj Skripte